In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel


In [ ]:
# Load BERT tokenizer and model
bert_model_name = 'aubmindlab/bert-base-arabertv2'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = TFBertModel.from_pretrained(bert_model_name)


In [ ]:
from arabert.preprocess import ArabertPreprocessor

arabert_prep = ArabertPreprocessor(model_name=bert_model_name)
input_text = " لاشتري فلوس مني والله اننو حقيقي"

# Tokenize input text
tokens = tokenizer(input_text, padding='max_length',
                   truncation=True, return_tensors='tf')
res = arabert_prep.preprocess(input_text)
print(tokens)

In [ ]:
print(type(bert_model))

In [ ]:
bert_output = bert_model(**tokens).last_hidden_state
print(bert_output)


In [ ]:
tokens['input_ids'].shape


In [ ]:
num_classes=2

# Add the pre-trained BERT layer
# Define inputs
input_ids = tf.keras.layers.Input(
    shape=(tokenizer.model_max_length,), dtype=tf.int32)

# BERT layer
bert_output = bert_model(input_ids).last_hidden_state

# Create the model
outputs = tf.keras.layers.GlobalAveragePooling1D()(
    bert_output)  # Example pooling layer
outputs = tf.keras.layers.Dense(units=128, activation='relu')(outputs)
outputs = tf.keras.layers.Dense(
    units=num_classes, activation='softmax')(outputs)

# Full model
model = tf.keras.Model(inputs=input_ids, outputs=outputs)

# Freeze the BERT layers
bert_model.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()



In [ ]:
print(tokenizer.model_max_length)


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ds = pd.read_csv('combined_cleaned_data.csv')
ds.head()

In [ ]:
ds = ds[['clean_text','spam']]
ds.head()

In [ ]:
ds['clean_text'] = ds['clean_text'].map(eval)

In [ ]:
ds['clean_text'] = ds['clean_text'].map(lambda x: " ".join(x)).map(lambda x : tokenizer(x, padding='max_length',
                                                                             truncation=True, return_tensors='tf'))



In [ ]:
clean_np = np.array(ds['clean_text'].map(lambda x: np.array(x.input_ids)).to_list())
clean_np = clean_np.reshape((90548,512))
print(clean_np.shape)

In [ ]:
x=tf.convert_to_tensor(clean_np)
y = tf.one_hot(tf.convert_to_tensor(ds['spam']), depth=num_classes)
print(x.shape)

In [ ]:
epochs = 10
history = model.fit(
    x=x,
    y=y,
    validation_split=0.2,
    epochs=epochs,
    verbose=1,
    batch_size=2)


In [ ]:
print(x)